In [1]:
import torch
from pytorch_quantization import tensor_quant

from torch import nn, outer

from pytorch_quantization import tensor_quant
import pytorch_quantization.nn as quant_nn
import pytorch_quantization
pytorch_quantization.__version__
from tqdm import tqdm

In [2]:
from datasets.yolo import YOLO
from engine.detector import Detector
from model.squeezedet import SqueezeDet, SqueezeDetWithLoss
from utils.config import Config
from utils.model import load_model
from PIL import Image
import numpy as np
import argparse
import os
from utils.misc import init_env

In [3]:
args = {'_': '_'}
cfg = Config().parse(args)

init_env(cfg) 
 
cfg.mode = 'eval'
cfg.exp_id = "test_lpd_sqdet_trt_calib"
cfg.dataset = 'lpr'
cfg.load_model = '../exp/alpr_det.pth' 
cfg.batch_size = 1
cfg.arch = 'squeezedet'
cfg.num_workers = 0
cfg.save_intervals = 1
cfg.debug = 2
cfg.load_pretrained = True

def update_exp_dir(cfg, exp_id):
    cfg.save_dir = os.path.join(cfg.exp_dir, exp_id)
    os.makedirs(cfg.save_dir, exist_ok=True)
    cfg.log_file = os.path.join(cfg.save_dir, 'training_logs.txt')
    os.remove(cfg.log_file) if os.path.exists(cfg.log_file) else None
    cfg.debug_dir = os.path.join(cfg.save_dir, 'debug')

update_exp_dir(cfg, cfg.exp_id)

Config().print(cfg)


arch                           squeezedet
batch_size                     1
bbox_loss_weight               20.0
chunk_sizes                    [32]
class_loss_weight              1.0
data_dir                       /media/gj_data/QAT_Squeezedet/SqueezeDet-PyTorch/data
dataset                        lpr
debug                          2
debug_dir                      /media/gj_data/QAT_Squeezedet/SqueezeDet-PyTorch/exp/test_lpd_sqdet_trt_calib/debug
device                         cuda
drift_prob                     1.0
dropout_prob                   0
exp_dir                        /media/gj_data/QAT_Squeezedet/SqueezeDet-PyTorch/exp
exp_id                         test_lpd_sqdet_trt_calib
flip_prob                      0.5
forbid_resize                  False
gpus                           [0]
gpus_str                       0
grad_norm                      0.5
keep_top_k                     64
load_model                     ../exp/alpr_det.pth
load_pretrained                True
log_file  

In [4]:
from engine.detector import Detector
from model.squeezedet import SqueezeDetWithLoss
from utils.model import load_model
from utils.misc import load_dataset

In [5]:
dataset = load_dataset(cfg.dataset)('val', cfg)
cfg = Config().update_dataset_info(cfg, dataset)
Config().print(cfg)

anchors                        [[  8.   8.   6.   5.]
 [  8.   8.  12.  10.]
 [  8.   8.  18.  10.]
 ...
 [248. 248.  18.  18.]
 [248. 248.  20.  24.]
 [248. 248.  30.  15.]]
anchors_per_grid               6
arch                           squeezedet
batch_size                     1
bbox_loss_weight               20.0
chunk_sizes                    [32]
class_loss_weight              1.0
class_names                    0
data_dir                       /media/gj_data/QAT_Squeezedet/SqueezeDet-PyTorch/data
dataset                        lpr
debug                          2
debug_dir                      /media/gj_data/QAT_Squeezedet/SqueezeDet-PyTorch/exp/test_lpd_sqdet_trt_calib/debug
device                         cuda
drift_prob                     1.0
dropout_prob                   0
exp_dir                        /media/gj_data/QAT_Squeezedet/SqueezeDet-PyTorch/exp
exp_id                         test_lpd_sqdet_trt_calib
flip_prob                      0.5
forbid_resize                 

### Vanilla Eval

In [6]:
from utils.misc import init_env
init_env(cfg)
from eval import eval
eval(cfg)

anchors                        [[  8.   8.   6.   5.]
 [  8.   8.  12.  10.]
 [  8.   8.  18.  10.]
 ...
 [248. 248.  18.  18.]
 [248. 248.  20.  24.]
 [248. 248.  30.  15.]]
anchors_per_grid               6
arch                           squeezedet
batch_size                     1
bbox_loss_weight               20.0
chunk_sizes                    [32]
class_loss_weight              1.0
class_names                    0
data_dir                       /media/gj_data/QAT_Squeezedet/SqueezeDet-PyTorch/data
dataset                        lpr
debug                          2
debug_dir                      /media/gj_data/QAT_Squeezedet/SqueezeDet-PyTorch/exp/test_lpd_sqdet_trt_calib/debug
device                         cuda
drift_prob                     1.0
dropout_prob                   0
exp_dir                        /media/gj_data/QAT_Squeezedet/SqueezeDet-PyTorch/exp
exp_id                         test_lpd_sqdet_trt_calib
flip_prob                      0.5
forbid_resize                 

mkdir: cannot create directory ‘/media/gj_data/QAT_Squeezedet/SqueezeDet-PyTorch/exp/test_lpd_sqdet_trt_calib/results/plot’: File exists


Loading detections...
  done.
Your evaluation results are available in /media/gj_data/QAT_Squeezedet/SqueezeDet-PyTorch/exp/test_lpd_sqdet_trt_calib/results
0_easy               0.894
0_moderate           0.894
0_hard               0.894
mAP                  0.894


sh: 1: gnuplot: not found
sh: 1: gnuplot: not found
Error: /undefinedfilename in (car_detection.eps)
Operand stack:

Execution stack:
   %interp_exit   .runexec2   --nostringval--   --nostringval--   --nostringval--   2   %stopped_push   --nostringval--   --nostringval--   --nostringval--   false   1   %stopped_push
Dictionary stack:
   --dict:731/1123(ro)(G)--   --dict:0/20(G)--   --dict:75/200(L)--
Current allocation mode is local
Last OS error: No such file or directory
GPL Ghostscript 9.50: Unrecoverable error, exit code 1
sh: 1: pdfcrop: not found


In [28]:
if cfg.mode=='eval':
        model = SqueezeDetWithLoss(cfg)
        model = load_model(model, cfg.load_model, cfg)


model.detect = True
detector = Detector(model, cfg)
print(detector.model)


I0902 22:06:11.508260 139837141516480 _utils.py:72] Input is fake quantized to 8 bits in QuantConv2d with axis None!
I0902 22:06:11.508705 139837141516480 _utils.py:75] Weight is fake quantized to 8 bits in QuantConv2d with axis 0!
I0902 22:06:11.509101 139837141516480 tensor_quantizer.py:101] Creating histogram calibrator
I0902 22:06:11.509482 139837141516480 tensor_quantizer.py:105] Creating Max calibrator
I0902 22:06:11.517278 139837141516480 _utils.py:72] Input is fake quantized to 8 bits in QuantConv2d with axis None!
I0902 22:06:11.517872 139837141516480 _utils.py:75] Weight is fake quantized to 8 bits in QuantConv2d with axis 0!
I0902 22:06:11.518282 139837141516480 tensor_quantizer.py:101] Creating histogram calibrator
I0902 22:06:11.518712 139837141516480 tensor_quantizer.py:105] Creating Max calibrator
I0902 22:06:11.526389 139837141516480 _utils.py:72] Input is fake quantized to 8 bits in QuantConv2d with axis None!
I0902 22:06:11.526901 139837141516480 _utils.py:75] Weight 

loaded model ../exp/alpr_det.pth, epoch 268
Model successfully loaded.
SqueezeDetWithLoss(
  (base): SqueezeDetBase(
    (quant): QuantStub()
    (dequant): DeQuantStub()
    (conv1): QuantConv2d(
      3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1)
      (_input_quantizer): TensorQuantizer(8bit fake per-tensor amax=dynamic calibrator=HistogramCalibrator scale=1.0 quant)
      (_weight_quantizer): TensorQuantizer(8bit fake axis=0 amax=dynamic calibrator=MaxCalibrator scale=1.0 quant)
    )
    (relu1): ReLU(inplace=True)
    (features): Sequential(
      (0): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
      (1): Fire(
        (squeeze): QuantConv2d(
          64, 16, kernel_size=(1, 1), stride=(1, 1)
          (_input_quantizer): TensorQuantizer(8bit fake per-tensor amax=dynamic calibrator=HistogramCalibrator scale=1.0 quant)
          (_weight_quantizer): TensorQuantizer(8bit fake axis=0 amax=dynamic calibrator=MaxCalibrator scale=1.0 quant)
  

In [20]:
from pytorch_quantization import nn as quant_nn
from pytorch_quantization import calib
from pytorch_quantization.tensor_quant import QuantDescriptor

from absl import logging
logging.set_verbosity(logging.INFO)  # Disable logging as they are too noisy in notebook

In [21]:
quant_desc_input = QuantDescriptor(calib_method='histogram')
quant_nn.QuantConv2d.set_default_quant_desc_input(quant_desc_input)
quant_nn.QuantLinear.set_default_quant_desc_input(quant_desc_input)

In [22]:
from pytorch_quantization import quant_modules
quant_modules.initialize()

### Calibration Functions

In [23]:
def collect_stats(model, data_loader, num_batches):
    """Feed data to the network and collect statistic"""

    # Enable calibrators
    for name, module in model.named_modules():
        if isinstance(module, quant_nn.TensorQuantizer):
            if module._calibrator is not None:
                module.disable_quant()
                module.enable_calib()
            else:
                module.disable()

    # for i, (image, _) in tqdm(enumerate(data_loader), total=num_batches):
    #     model(image.cuda())
    #     if i >= num_batches:
    #         break
    # since lpr dataloader return a dict, so writing it in this way
    for i, (batch) in tqdm(enumerate(data_loader), total=num_batches):
        for k in batch:
                if 'image_meta' not in k:
                    batch[k] = batch[k].to(device=cfg.device, non_blocking=True)
        model(batch)
        if i >= num_batches:
            break

    # Disable calibrators
    for name, module in model.named_modules():
        if isinstance(module, quant_nn.TensorQuantizer):
            if module._calibrator is not None:
                module.enable_quant()
                module.disable_calib()
            else:
                module.enable()
            
def compute_amax(model, **kwargs):
    # Load calib result
    for name, module in model.named_modules():
        if isinstance(module, quant_nn.TensorQuantizer):
            if module._calibrator is not None:
                if isinstance(module._calibrator, calib.MaxCalibrator):
                    module.load_calib_amax()
                else:
                    module.load_calib_amax(**kwargs)
#             print(F"{name:40}: {module}")
    model.cuda()

## Make Dataset and DataLoader

In [5]:
import os
import subprocess

import numpy as np
import skimage.io

from datasets.base import BaseDataset
from utils.boxes import generate_anchors
from PIL import Image
from torchvision.datasets.folder import default_loader


class LPR_CalibTRT(BaseDataset):
    def __init__(self, phase, cfg):
        super(LPR_CalibTRT, self).__init__(phase, cfg)

        # self.input_size = (128, 128)  # (height, width), both dividable by 16
        #this model for sadaia works with (256,256)
        self.input_size = (256, 256)  # (height, width), both dividable by 16
        self.class_names = ('0')
        # real_filtered mean and std
        # self.rgb_mean = np.array([94.87347, 96.89165, 94.70493], dtype=np.float32).reshape(1, 1, 3)
        # self.rgb_std = np.array([53.869507, 53.936283, 55.2807], dtype=np.float32).reshape(1, 1, 3)
        
        # real_filtered plus all_sites_seatbelt mean and std
        # self.rgb_mean = np.array([104.90631, 105.41336, 104.70162], dtype=np.float32).reshape(1, 1, 3)
        # self.rgb_std = np.array([50.69564, 49.60443, 50.158844], dtype=np.float32).reshape(1, 1, 3)

        self.rgb_mean = np.array([97.631615, 98.70732, 98.41285], dtype=np.float32).reshape(1, 1, 3)
        self.rgb_std = np.array([52.766678, 52.63513, 52.348827], dtype=np.float32).reshape(1, 1, 3)

        self.num_classes = len(self.class_names)
        self.class_ids_dict = {cls_name: cls_id for cls_id, cls_name in enumerate(self.class_names)}

        self.data_dir = os.path.join(cfg.data_dir, 'lpr_crop/merged_data')
        self.sample_ids, self.sample_set_path = self.get_sample_ids()
        self.grid_size = tuple(x // cfg.stride for x in self.input_size)  # anchors grid 
        # self.anchors_seed = np.array([[ 29, 17], [46, 32], [69, 52],
        #                                 [109, 68], [84, 127], [155, 106], 
        #                                 [255, 145], [183, 215], [371, 221]], dtype=np.float32) ## real_filtered anchors
        
        # self.anchors_seed = np.array( [[ 32, 20], [ 61, 42], [ 59, 97],
        #                                 [103, 66], [122, 114], [183, 96],
        #                                 [160, 152], [211, 201], [343, 205]], dtype=np.float32) ## real_filtered plus all_sites_seatbelt anchors

        # self.anchors_seed = np.array([[6, 5], [12, 10], [18, 10], [18, 18], [20, 24], [30, 15]], dtype=np.float32)
        self.anchors_seed = np.array([[3, 3], [6, 5], [9, 5], [10, 9], [10, 12], [15, 8]], dtype=np.float32)
        # self.anchors = generate_anchors(self.grid_size, self.input_size, self.anchors_seed)
        self.anchors = torch.load("../exp/alpr_det_anchors_256_exp_8.pt")
        self.anchors_per_grid = self.anchors_seed.shape[0]
        self.num_anchors = self.anchors.shape[0]

        self.results_dir = os.path.join(cfg.save_dir, 'results')

    def get_sample_ids(self):
        #a dirty duct tape to load preprocessing of val phase but load dataset for train phase for trt calib
        sample_set_name = 'train.txt' if self.phase == 'train' \
            else 'train.txt' if self.phase == 'val' \
            else 'trainval.txt' if self.phase == 'trainval' \
            else None

        sample_ids_path = os.path.join(self.data_dir, sample_set_name)
        with open(sample_ids_path, 'r') as fp:
            sample_ids = fp.readlines()
        sample_ids = tuple(x.strip() for x in sample_ids)

        return sample_ids, sample_ids_path

    def load_image(self, index):
        image_id = self.sample_ids[index]
        image_path = os.path.join(self.data_dir, 'images', image_id + '.png')
        image = default_loader(image_path)
        if image.mode == 'L':
            image = image.convert('RGB')
        image = np.array(image).astype(np.float32)
        # image = skimage.io.imread(image_path).astype(np.float32)
        return image, image_id

    def load_annotations(self, index):
        ann_id = self.sample_ids[index]
        ann_path = os.path.join(self.data_dir, 'labels', ann_id + '.txt')
        with open(ann_path, 'r') as fp:
            annotations = fp.readlines()

        annotations = [ann.strip().split(' ') for ann in annotations]
        class_ids, boxes = [], []
        for ann in annotations:
            if ann[0] not in self.class_names:
                continue
            class_ids.append(self.class_ids_dict[ann[0]])
            box = [float(x) for x in ann[4:8]]
            # if box[2] <= 0:
            #     box[2] = 0.00001
            # if box[3] <= 0:
            #     box[3] = 0.00001
            boxes.append(box)

        class_ids = np.array(class_ids, dtype=np.int16)
        boxes = np.array(boxes, dtype=np.float32)
        if len(boxes):
            return class_ids, boxes
        boxes = None
        return class_ids, boxes

    # ========================================
    #                evaluation
    # ========================================

    def save_results(self, results):
        txt_dir = os.path.join(self.results_dir, 'data')
        os.makedirs(txt_dir, exist_ok=True)

        for res in results:
            txt_path = os.path.join(txt_dir, res['image_meta']['image_id'] + '.txt')
            if 'class_ids' not in res:
                with open(txt_path, 'w') as fp:
                    fp.write('')
                continue

            num_boxes = len(res['class_ids'])
            with open(txt_path, 'w') as fp:
                for i in range(num_boxes):
                    class_name = self.class_names[res['class_ids'][i]].lower()
                    score = res['scores'][i]
                    bbox = res['boxes'][i, :]
                    line = '{} -1 -1 0 {:.2f} {:.2f} {:.2f} {:.2f} 0 0 0 0 0 0 0 {:.3f}\n'.format(
                            class_name, *bbox, score)
                    fp.write(line)

    def evaluate(self):
        kitti_eval_tool_path = os.path.join(self.cfg.root_dir, 'src/utils/kitti-eval/cpp/evaluate_object')
        cmd = '{} {} {} {} {}'.format(kitti_eval_tool_path,
                                      os.path.join(self.data_dir),
                                      self.sample_set_path,
                                      self.results_dir,
                                      len(self.sample_ids))

        print (cmd)
        status = subprocess.call(cmd, shell=True)

        aps = {}
        for class_name in self.class_names:
            map_path = os.path.join(self.results_dir, 'stats_{}_ap.txt'.format(class_name.lower()))
            if os.path.exists(map_path):
                with open(map_path, 'r') as f:
                    lines = f.readlines()
                _aps = [float(line.split('=')[1].strip()) for line in lines]
            else:
                _aps = [0., 0., 0.]

            aps[class_name + '_easy'] = _aps[0]
            aps[class_name + '_moderate'] = _aps[1]
            aps[class_name + '_hard'] = _aps[2]

        aps['mAP'] = sum(aps.values()) / len(aps)

        return aps

In [6]:
lpr_calib_dataset = LPR_CalibTRT('val', cfg)
calib_data_loader = torch.utils.data.DataLoader(lpr_calib_dataset,
                                                  batch_size=cfg.batch_size,
                                                  num_workers=cfg.num_workers,
                                                  pin_memory=False)

In [17]:
print (len(lpr_calib_dataset))

14068


### Compute stats and Calibrate on Different schemes

In [29]:
num_batches = int(len(lpr_calib_dataset)/cfg.batch_size)
print (num_batches)

detector.model.eval()
detector.model.cuda()
with torch.no_grad():
    collect_stats(detector.model, calib_data_loader, num_batches=2000)
    # compute_amax(detector.model, method="percentile", percentile=99.99)

I0902 22:06:29.795573 139837141516480 tensor_quantizer.py:183] Disable `quant` stage.
I0902 22:06:29.796108 139837141516480 tensor_quantizer.py:179] Enable HistogramCalibrator
I0902 22:06:29.796442 139837141516480 tensor_quantizer.py:183] Disable `quant` stage.
I0902 22:06:29.796764 139837141516480 tensor_quantizer.py:179] Enable MaxCalibrator
I0902 22:06:29.797045 139837141516480 tensor_quantizer.py:183] Disable `quant` stage.
I0902 22:06:29.797311 139837141516480 tensor_quantizer.py:179] Enable HistogramCalibrator
I0902 22:06:29.797579 139837141516480 tensor_quantizer.py:183] Disable `quant` stage.
I0902 22:06:29.798238 139837141516480 tensor_quantizer.py:179] Enable MaxCalibrator
I0902 22:06:29.798545 139837141516480 tensor_quantizer.py:183] Disable `quant` stage.
I0902 22:06:29.798936 139837141516480 tensor_quantizer.py:179] Enable HistogramCalibrator
I0902 22:06:29.799295 139837141516480 tensor_quantizer.py:183] Disable `quant` stage.
I0902 22:06:29.799603 139837141516480 tensor_q

14068


100%|██████████| 2000/2000 [03:49<00:00,  8.71it/s]
I0902 22:10:19.567061 139837141516480 tensor_quantizer.py:187] Enable `quant` stage.
W0902 22:10:19.567570 139837141516480 tensor_quantizer.py:173] Disable HistogramCalibrator
I0902 22:10:19.567900 139837141516480 tensor_quantizer.py:187] Enable `quant` stage.
W0902 22:10:19.568234 139837141516480 tensor_quantizer.py:173] Disable MaxCalibrator
I0902 22:10:19.568527 139837141516480 tensor_quantizer.py:187] Enable `quant` stage.
W0902 22:10:19.569022 139837141516480 tensor_quantizer.py:173] Disable HistogramCalibrator
I0902 22:10:19.569306 139837141516480 tensor_quantizer.py:187] Enable `quant` stage.
W0902 22:10:19.569953 139837141516480 tensor_quantizer.py:173] Disable MaxCalibrator
I0902 22:10:19.570300 139837141516480 tensor_quantizer.py:187] Enable `quant` stage.
W0902 22:10:19.570678 139837141516480 tensor_quantizer.py:173] Disable HistogramCalibrator
I0902 22:10:19.571048 139837141516480 tensor_quantizer.py:187] Enable `quant` st

In [30]:
print (detector.model)

SqueezeDetWithLoss(
  (base): SqueezeDetBase(
    (quant): QuantStub()
    (dequant): DeQuantStub()
    (conv1): QuantConv2d(
      3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1)
      (_input_quantizer): TensorQuantizer(8bit fake per-tensor amax=dynamic calibrator=HistogramCalibrator scale=1.0 quant)
      (_weight_quantizer): TensorQuantizer(8bit fake axis=0 amax=dynamic calibrator=MaxCalibrator scale=1.0 quant)
    )
    (relu1): ReLU(inplace=True)
    (features): Sequential(
      (0): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
      (1): Fire(
        (squeeze): QuantConv2d(
          64, 16, kernel_size=(1, 1), stride=(1, 1)
          (_input_quantizer): TensorQuantizer(8bit fake per-tensor amax=dynamic calibrator=HistogramCalibrator scale=1.0 quant)
          (_weight_quantizer): TensorQuantizer(8bit fake axis=0 amax=dynamic calibrator=MaxCalibrator scale=1.0 quant)
        )
        (expand1x1): QuantConv2d(
          16, 64, kernel_size

In [31]:
from datasets.lpr import LPR

eval_dataset = LPR('val', cfg)
eval_dataset.phase

'val'

In [32]:
with torch.no_grad():
    for method in ["percentile", "mse", "entropy"]:
        percentile = 99.99
        print(F" ############  {method} calibration ######################")
        if(method == 'percentile'):
            compute_amax(detector.model, method=method, percentile = percentile)
        else:
            compute_amax(detector.model, method=method)
        update_exp_dir(cfg, f"trt_calib_lpd_sadaia_sqdet_{method}")
        results = detector.detect_dataset(eval_dataset, cfg)
        eval_dataset.save_results(results)
        aps = eval_dataset.evaluate()
        for k, v in aps.items():
                print('{:<20} {:.3f}'.format(k, v))
        torch.save(detector.model.state_dict(), os.path.join(cfg.save_dir, f"quant_lp_squeezedet_sadaia_{method}.pth"))

W0902 22:11:53.662745 139837141516480 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0902 22:11:53.663644 139837141516480 tensor_quantizer.py:238] Call .cuda() if running on GPU after loading calibrated amax.
W0902 22:11:53.664341 139837141516480 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([64, 1, 1, 1]).
W0902 22:11:53.664942 139837141516480 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0902 22:11:53.665276 139837141516480 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([16, 1, 1, 1]).
W0902 22:11:53.665915 139837141516480 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0902 22:11:53.666280 139837141516480 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([64, 1, 1, 1]).
W0902 22:11:53.666913 139837141516480 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0902 22:11:53.667380 139837141516480 tensor_quantizer.py:237] Load calibrated amax, shape=to

 ############  percentile calibration ######################
eval: [0/1530] | data 0.102s | net 0.050s
eval: [20/1530] | data 0.120s | net 0.029s
eval: [40/1530] | data 0.088s | net 0.031s
eval: [60/1530] | data 0.013s | net 0.036s
eval: [80/1530] | data 0.015s | net 0.049s
eval: [100/1530] | data 0.031s | net 0.084s
eval: [120/1530] | data 0.013s | net 0.028s
eval: [140/1530] | data 0.033s | net 0.089s
eval: [160/1530] | data 0.007s | net 0.031s
eval: [180/1530] | data 0.006s | net 0.021s
eval: [200/1530] | data 0.014s | net 0.048s
eval: [220/1530] | data 0.048s | net 0.144s
eval: [240/1530] | data 0.017s | net 0.056s
eval: [260/1530] | data 0.007s | net 0.029s
eval: [280/1530] | data 0.002s | net 0.019s
eval: [300/1530] | data 0.019s | net 0.043s
eval: [320/1530] | data 0.005s | net 0.027s
eval: [340/1530] | data 0.012s | net 0.043s
eval: [360/1530] | data 0.024s | net 0.078s
eval: [380/1530] | data 0.009s | net 0.038s
eval: [400/1530] | data 0.037s | net 0.066s
eval: [420/1530] | da

mkdir: cannot create directory ‘/home/gj/hazen/QAT_Squeezedet/SqueezeDet-PyTorch/exp/test_lpd_sqdet_trt_calib/results/plot’: File exists
sh: 1: gnuplot: not found
sh: 1: gnuplot: not found
Error: /undefinedfilename in (car_detection.eps)
Operand stack:

Execution stack:
   %interp_exit   .runexec2   --nostringval--   --nostringval--   --nostringval--   2   %stopped_push   --nostringval--   --nostringval--   --nostringval--   false   1   %stopped_push
Dictionary stack:
   --dict:956/1684(ro)(G)--   --dict:0/20(G)--   --dict:77/200(L)--
Current allocation mode is local
Last OS error: No such file or directory
GPL Ghostscript 9.26: Unrecoverable error, exit code 1
sh: 1: pdfcrop: not found


Loading detections...
  done.
Your evaluation results are available in /home/gj/hazen/QAT_Squeezedet/SqueezeDet-PyTorch/exp/test_lpd_sqdet_trt_calib/results
0_easy               0.893
0_moderate           0.893
0_hard               0.893
mAP                  0.893
 ############  mse calibration ######################


W0902 22:13:28.884643 139837141516480 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0902 22:13:28.885464 139837141516480 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([64, 1, 1, 1]).
W0902 22:13:31.608424 139837141516480 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0902 22:13:31.609170 139837141516480 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([16, 1, 1, 1]).
W0902 22:13:36.137655 139837141516480 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0902 22:13:36.138330 139837141516480 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([64, 1, 1, 1]).
W0902 22:13:40.481978 139837141516480 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0902 22:13:40.482854 139837141516480 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([64, 1, 1, 1]).
W0902 22:13:43.874020 139837141516480 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Siz

...

W0902 22:16:03.796549 139837141516480 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0902 22:16:03.797331 139837141516480 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([64, 1, 1, 1]).
W0902 22:16:43.217256 139837141516480 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0902 22:16:43.223551 139837141516480 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([16, 1, 1, 1]).
W0902 22:18:04.357078 139837141516480 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0902 22:18:04.363376 139837141516480 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([64, 1, 1, 1]).
W0902 22:19:25.722566 139837141516480 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0902 22:19:25.728924 139837141516480 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([64, 1, 1, 1]).
W0902 22:20:03.887575 139837141516480 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Siz

eval: [0/1530] | data 0.005s | net 0.022s
eval: [20/1530] | data 0.005s | net 0.027s
eval: [40/1530] | data 0.020s | net 0.031s
eval: [60/1530] | data 0.101s | net 0.020s
eval: [80/1530] | data 0.016s | net 0.050s
eval: [100/1530] | data 0.146s | net 0.085s
eval: [120/1530] | data 0.005s | net 0.029s
eval: [140/1530] | data 0.032s | net 0.089s
eval: [160/1530] | data 0.007s | net 0.030s
eval: [180/1530] | data 0.002s | net 0.021s
eval: [200/1530] | data 0.025s | net 0.049s
eval: [220/1530] | data 0.048s | net 0.145s
eval: [240/1530] | data 0.018s | net 0.056s
eval: [260/1530] | data 0.006s | net 0.029s
eval: [280/1530] | data 0.002s | net 0.019s
eval: [300/1530] | data 0.029s | net 0.045s
eval: [320/1530] | data 0.006s | net 0.027s
eval: [340/1530] | data 0.012s | net 0.042s
eval: [360/1530] | data 0.098s | net 0.072s
eval: [380/1530] | data 0.009s | net 0.035s
eval: [400/1530] | data 0.049s | net 0.066s
eval: [420/1530] | data 0.004s | net 0.024s
eval: [440/1530] | data 0.038s | net 0

mkdir: cannot create directory ‘/home/gj/hazen/QAT_Squeezedet/SqueezeDet-PyTorch/exp/test_lpd_sqdet_trt_calib/results/plot’: File exists


Loading detections...
  done.
Your evaluation results are available in /home/gj/hazen/QAT_Squeezedet/SqueezeDet-PyTorch/exp/test_lpd_sqdet_trt_calib/results
0_easy               0.892
0_moderate           0.892
0_hard               0.892
mAP                  0.892


sh: 1: gnuplot: not found
sh: 1: gnuplot: not found
Error: /undefinedfilename in (car_detection.eps)
Operand stack:

Execution stack:
   %interp_exit   .runexec2   --nostringval--   --nostringval--   --nostringval--   2   %stopped_push   --nostringval--   --nostringval--   --nostringval--   false   1   %stopped_push
Dictionary stack:
   --dict:956/1684(ro)(G)--   --dict:0/20(G)--   --dict:77/200(L)--
Current allocation mode is local
Last OS error: No such file or directory
GPL Ghostscript 9.26: Unrecoverable error, exit code 1
sh: 1: pdfcrop: not found


In [ ]:
# print (detector.model)

### Add scales to the quantized state dict

In [ ]:
dict = torch.load("../exp/trt_calib_lpd_sadaia_sqdet_percentile/quant_lp_squeezedet_sadaia_percentile.pth")
print (dict.keys())



In [37]:
# print (alpr)
def get_scales_input_per_tensor(amax, _num_bits = 8 , _unsigned = False):
    bound = (1 << (_num_bits - 1 + int(_unsigned))) - 1    
    zero_point_per_tensor = 0 #since symmertric quant
    scale_per_tensor = amax / bound
    return scale_per_tensor, zero_point_per_tensor

def get_scales_weight_per_channel(amax, _num_bits = 8, _unsigned = False):
    bound = (1 << (_num_bits - 1 + int(_unsigned))) - 1
    amax_sequeeze = amax.squeeze().detach()
    scale_per_channel =  amax_sequeeze/bound
    zero_point_per_channel = torch.zeros_like(scale_per_channel, dtype=torch.int32).data
    quant_axis = list(amax.shape).index(list(amax_sequeeze.shape)[0])
    return scale_per_channel, zero_point_per_channel


In [38]:
lp_det_sadaia_dict_with_scales = {}

for key in dict:
    if key.find('_input_quantizer._amax') != -1:
        amax = dict[key]
        scales, zero_point = get_scales_input_per_tensor(amax)
        new_key = key + '.scales'
        lp_det_sadaia_dict_with_scales[new_key] = scales
    elif key.find('_weight_quantizer._amax') != -1:
        amax = dict[key]
        scales, zero_point = get_scales_weight_per_channel(amax)
        new_key = key + '.scales'
        lp_det_sadaia_dict_with_scales[new_key] = scales
    
    lp_det_sadaia_dict_with_scales[key] = dict[key]

print (lp_det_sadaia_dict_with_scales.keys())

dict_keys(['base.conv1.weight', 'base.conv1.bias', 'base.conv1._input_quantizer._amax.scales', 'base.conv1._input_quantizer._amax', 'base.conv1._weight_quantizer._amax.scales', 'base.conv1._weight_quantizer._amax', 'base.features.1.squeeze.weight', 'base.features.1.squeeze.bias', 'base.features.1.squeeze._input_quantizer._amax.scales', 'base.features.1.squeeze._input_quantizer._amax', 'base.features.1.squeeze._weight_quantizer._amax.scales', 'base.features.1.squeeze._weight_quantizer._amax', 'base.features.1.expand1x1.weight', 'base.features.1.expand1x1.bias', 'base.features.1.expand1x1._input_quantizer._amax.scales', 'base.features.1.expand1x1._input_quantizer._amax', 'base.features.1.expand1x1._weight_quantizer._amax.scales', 'base.features.1.expand1x1._weight_quantizer._amax', 'base.features.1.expand3x3.weight', 'base.features.1.expand3x3.bias', 'base.features.1.expand3x3._input_quantizer._amax.scales', 'base.features.1.expand3x3._input_quantizer._amax', 'base.features.1.expand3x3._

In [39]:
torch.save(lp_det_sadaia_dict_with_scales, "../exp/quant_lp_det_sadaia_with_scales.pth")